[UMAP](https://umap-learn.readthedocs.io/en/latest/) is a powerful dimensionality reduction tool which NVIDIA recently ported GPUs with a python interface.  In this notebook we will demostrate basic usage, plotting, and timing comparisons between the CUDA (GPU) version of UMAP and the original CPU version of UMAP 

In [ ]:
import os
import pandas as pd
import numpy as np

# CPU UMAP
import umap
# os.environ['NUMBA_NUM_THREADS'] = '1'
# os.environ['NUMBA_DISABLE_JIT'] = '1'

# libraries for scoring/clustering
from sklearn.manifold.t_sne import trustworthiness

# GPU UMAP
import cudf
from cuml.manifold.umap import UMAP as cumlUMAP

# plotting
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='white', rc={'figure.figsize':(25, 12.5)})

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings("ignore") 

We are going to work with the [fashion mnist](https://github.com/zalandoresearch/fashion-mnist) data set.  This is a dataset consisting of 70,000 28x28 grayscale images of clothing.

In [ ]:
if not os.path.exists('fashion_mnist'):
    !git clone https://github.com/zalandoresearch/fashion-mnist.git fashion_mnist

In [ ]:
# https://github.com/zalandoresearch/fashion-mnist/blob/master/utils/mnist_reader.py
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [ ]:
train, train_labels = load_mnist('fashion_mnist/data/fashion', kind='train')
test, test_labels = load_mnist('fashion_mnist/data/fashion', kind='t10k')
data = np.array(np.vstack([train, test]), dtype=np.float64) / 255.0
target = np.array(np.hstack([train_labels, test_labels]))

There are 60000 training images and 10000 test images

In [ ]:
f"Train shape: {train.shape} and Test Shape: {test.shape}"

In [ ]:
train[0].shape

As mentioned previously, each row in the train matrix is an image

In [ ]:
# display a Nike? sneaker
pixels = train[0].reshape((28, 28))
plt.imshow(pixels, cmap='gray')

There is cost with moving data between host memory and device memory (GPU memory) and we will include that core when comparing speeds

In [ ]:
%%time
record_data = (('fea%d'%i, data[:,i]) for i in range(data.shape[1]))
gdf = cudf.DataFrame(record_data)

In [ ]:
%%time
embedding = umap.UMAP(n_neighbors=5, init="spectral").fit_transform(data)

In [ ]:
%%time
g_embedding = cumlUMAP(n_neighbors=5, init="spectral").fit_transform(gdf)

In [ ]:
print(f'Size of data in memory: {data.nbytes / 1e6} MB')

`gdf` is a GPU backed dataframe -- all the data is stored in the device memory of the GPU.  With the data converted, we can apply the `cumlUMAP` the same inputs as we do for the standard UMAP.  Additionally, it should be noted that within cuml, [FAISS] https://github.com/facebookresearch/faiss) is used for extremely fast kNN and it's limited to single precision.  `cumlUMAP` will automatically downcast to `float32` when needed.

In [ ]:
%%timeit
embedding = umap.UMAP(n_neighbors=5, init="spectral").fit_transform(data)

Compute Runtimes of UMAP training (lower is better):
- cumlUMAP (GPU): 6.8
- UMAP (CPU): 60 seconds

cumlMAP is ~10x speed up over UMAP CPU.  We should also qualitatively compare output accuracy.  We can do this easily by plotting the embeddings and noting simlarites in the plot

In [ ]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot']

In [ ]:
g_numpy_embedding = g_embedding.to_pandas().values

Just as the original author of UMAP, Leland McInnes, states in the [UMAP docs](https://umap-learn.readthedocs.io/en/latest/supervised.html), we can plot the results and show the separation between the various classes defined above.

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10))
plt.scatter(g_numpy_embedding[:, 1], g_numpy_embedding[:,0], s=0.3, c=target, cmap='Spectral', alpha=1.0)
plt.setp(ax, xticks=[], yticks=[])
cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(classes)
plt.title('Fashion MNIST Embedded via cumlUMAP');

And side-by-side we can see that the separation is very similar between both GPU and CPU versions of UMAP.  Both cluster Ankle Boot, Sneaker, and Sandal together (Green/Yellow/Purple), both separate Trousers and Bags entirely, and both have T-shirts, Shirts, Coats, etc. all mixed toether.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
ax[0].scatter(g_numpy_embedding[:,1], g_numpy_embedding[:,0], s=0.3, c=target, cmap='Spectral', alpha=1.0)
im = ax[1].scatter(embedding[:,1], embedding[:,0], s=0.3, c=target, cmap='Spectral', alpha=1.0)
ax[0].set_title('Fashion MNIST Embedded via cumlUMAP');
ax[1].set_title('Fashion MNIST Embedded via UMAP');

fig.subplots_adjust(right=0.8)
cax,kw = mpl.colorbar.make_axes([a for a in ax.flat])
cbar = plt.colorbar(im, cax=cax, **kw)
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(classes)

Additionally, we can also quanititaviely compare the perfomance of `cumlUMAP` (GPU UMAP) to the reference/original implementation (CPU UMAP) using the [trustworthiness score](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/manifold/t_sne.py#L395).  From the docstring:

> Trustworthiness expresses to what extent the local structure is retained.  The trustworthiness is within [0, 1].


Like `t-SNE`, UMAP tries to capture both global and local structure and thus, we can apply the `trustworthiness` of the `embedding/g_embedding` data against the original input.  With a higher score we are demonstrating that the algorithm does a better and better job of local structure retention.  As [Corey Nolet](https://github.com/cjnolet) notes:
> Algorithms like UMAP aim to preserve local neighborhood structure and so measuring this property (trustworthiness) measures the algorithm's performance.

In [ ]:
%%time
# CPU VERSION
trust_cpu = trustworthiness(data, embedding, 5)

In [ ]:
%%time
# GPU VERSION
trust_gpu = trustworthiness(data, g_embedding, 5)

In [ ]:
print(f'CPU trustworthiness {trust_cpu} GPU trustworthiness: {trust_gpu}')

Scoring ~97% shows the GPU implementation is comparable to the original CPU implementation and the training time was ~9.5X faster

In [ ]:
embedding = umap.UMAP(n_neighbors=5, init="spectral").fit_transform(data)
record_data = (('fea%d'%i, data[:,i]) for i in range(data.shape[1]))
gdf = cudf.DataFrame(record_data)
g_embedding = cumlUMAP(n_neighbors=5, init="spectral").fit_transform(gdf)